# Search Demo Corpus

We'll now continue the example with a recipe from the Vectara website where we search our employee handbook.

See here for the REST version of this: https://docs.vectara.com/docs/api-recipes

NB: try --no-cache-dir for pip install for new versions

In [1]:
%pip install -q vectara-skunk-client==0.4.11

Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging

logging.basicConfig(format='%(asctime)s %(name)-20s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

In [3]:
from vectara.client.core import Factory

client = Factory().build()

08:54:09 +1100 Factory              INFO:initializing builder
08:54:09 +1100 Factory              INFO:Factory will load configuration from home directory
08:54:09 +1100 HomeConfigLoader     INFO:Loading configuration from users home directory [C:\Users\david]
08:54:09 +1100 HomeConfigLoader     INFO:Loading default configuration [default]
08:54:09 +1100 HomeConfigLoader     INFO:Parsing config
08:54:09 +1100 root                 INFO:We are processing authentication type [OAuth2]
08:54:09 +1100 OAuthUtil            INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
08:54:09 +1100 OAuthUtil            INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
08:54:09 +1100 root                 INFO:initializing Client


In [4]:
admin_service = client.admin_service

corpora = admin_service.list_corpora("02-employee-handbook")

if len(corpora) >= 1:
    for corpus in corpora:
        admin_service.delete_corpus(corpus.id)
else:
    logging.info("No existing corpus with the name client-example")

08:54:11 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:54:11.657893
08:54:11 +1100 OAuthUtil            INFO:First time requesting token, authenticating
08:54:12 +1100 OAuthUtil            INFO:Received OAuth token, will expire [01/13/2024, 09:54:12]
08:54:12 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100052]
08:54:12 +1100 RequestUtil          INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
08:54:13 +1100 root                 INFO:No existing corpus with the name client-example


## Create Our Demo Corpus
We now create our new corpus called, "02-employee-handbook" where we'll upload our documents for this test.

In [5]:
create_corpus_result = admin_service.create_corpus("02-employee-handbook", description="Example Corpus for use from Jupyter")
logging.info(f"New corpus created {create_corpus_result}")
corpus_id = create_corpus_result.corpusId

08:54:16 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:54:16.175940
08:54:16 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:54:12
08:54:16 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100052]
08:54:16 +1100 RequestUtil          INFO:URL for operation create-corpus is: https://api.vectara.io/v1/create-corpus
08:54:18 +1100 AdminService         INFO:Created new corpus with 243
08:54:18 +1100 root                 INFO:New corpus created CreateCorpusResponse(corpusId=243, status=Status(code=<StatusCode.OK: 0>, statusDetail='Corpus Created', cause=None))


## Upload Handbook to Corpus
We now upload our PDF to the corpus which will parse and then encode the document, storing the embeddings in our vector database.

In [6]:
from pathlib import Path

handbook_path = Path("resources/vectara_employee_handbook.pdf")
indexer_service = client.indexer_service
indexer_service.upload(corpus_id, handbook_path)

08:54:19 +1100 IndexerService       INFO:Headers: {"c": "1623270172", "o": "243"}
08:54:19 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:54:19.081316
08:54:19 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:54:12
08:54:19 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100052]
vectara_employee_handbook.pdf: 52.7kB [00:03, 14.9kB/s]                                                              


UploadDocumentResponse(response=UploadDocumentResponseInner(status=None, quotaConsumed=StorageQuota(numChars='9215', numMetadataChars='4903')), document=None)

In [7]:
query_service = client.query_service
query = "At Vectara, Can I bring any birds to the Vectara Office?"
response = query_service.query(query, corpus_id)

08:54:22 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:54:22.713223
08:54:22 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:54:12
08:54:22 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100052]
08:54:22 +1100 RequestUtil          INFO:URL for operation query is: https://api.vectara.io/v1/query


## Show Result in Markdown
We'll now use the utility method "renderMarkdown" to render the Python dict

In [8]:
from vectara.client.util import render_markdown
from IPython.display import display, Markdown
rendered = render_markdown(query, response)
display(Markdown(rendered))


# Query: At Vectara, Can I bring any birds to the Vectara Office?

Yes, you can bring birds to the Vectara office. The company encourages the presence of birds in their workspace as a symbol of their belief in soaring to new heights [1]. However, it's important to note that common household pets like cats and dogs are not permitted at the Vectara campuses [2]. The company's unique pet policy is part of their approach to thinking outside the box and breaking the mold [2][5]. Please refer to the Vectara Employee Handbook for more details on their pet policy [3].

 1. It's a celebration of the
unconventional, the daring, and the extraordinary that deﬁnes Vectara. 7.2.3: The Avian Exception - Birds of a Feather <b>At Vectara, we believe in soaring to new heights, and what better way to embody that spirit than
by allowing birds into our workspace?</b> Yes, you read that right! Birds are not only permitted but
encouraged. *score: 0.89985037, doc-id: vectara_employee_handbook.pdf*
 2. 7.2 - Company Pet Policy 7.2.1: The Usual Suspects - Cats & Dogs <b>We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses.</b> Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo. Cats and dogs are wonderful, but we believe in taking a walk on the
wild side. *score: 0.8140942, doc-id: vectara_employee_handbook.pdf*
 3. Employee Handbook - Company Pet Policy Vectara Employee Handbook <b>This employee handbook covers all the essential information to make you successful at
Vectara.</b> 7.2 - Company Pet Policy 7.2.1: The Usual Suspects - Cats & Dogs *score: 0.8071276, doc-id: vectara_employee_handbook.pdf*
 4. It's our
way of honoring the traditional bond between humans and their beloved pets while maintaining
our commitment to the extraordinary. While we adore cats and dogs, we must respectfully ask that they remain at home, where they
can continue their secret missions, philosophical ponderings, and adorable antics. <b>At Vectara,
we're forging a new path, one that requires us to leave behind the familiar and embrace the wild
unknown.</b> Rest assured, your feline and canine companions will be cheering you on from the
comfort of their favorite sunlit windows and cozy dog beds. 7.2.2: The Exotic Exceptions - Unleashing the Unusual *score: 0.8032716, doc-id: vectara_employee_handbook.pdf*
 5. Rest assured, your feline and canine companions will be cheering you on from the
comfort of their favorite sunlit windows and cozy dog beds. 7.2.2: The Exotic Exceptions - Unleashing the Unusual <b>At Vectara, we pride ourselves on thinking outside the box, and our pet policy is no exception.</b> While common household pets like cats and dogs are not permitted, we welcome a select group
of exotic creatures that reﬂect our daring and innovative spirit. Here's a closer look at our unique
approach: *score: 0.796098, doc-id: vectara_employee_handbook.pdf*
 6. 7.2.1: The Usual Suspects - Cats & Dogs We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses. <b>Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo.</b> Cats and dogs are wonderful, but we believe in taking a walk on the
wild side. Besides, we've heard that cats are secretly plotting world domination, and we can't
have that in our innovation hub! *score: 0.7942082, doc-id: vectara_employee_handbook.pdf*
 7. ○   Employee safety and animal welfare are our top priorities. Adoption Programs <b>The Conservation Connection: Vectara is committed to wildlife conservation.</b> Our exotic pet
policy is aligned with global conservation efforts, and we actively support initiatives to protect
endangered species. Our "Adopt a Rhino" program is a proud example. *score: 0.79056066, doc-id: vectara_employee_handbook.pdf*
 8. Employee Handbook - Company Pet Policy <b>Vectara Employee Handbook</b> This employee handbook covers all the essential information to make you successful at
Vectara. 7.2 - Company Pet Policy *score: 0.7843369, doc-id: vectara_employee_handbook.pdf*
 9. Our annual "Bear Hug Day" is a heartwarming tradition that fosters camaraderie both in
       the oﬃce and in the ICU. On-site Mini Zoo <b>●   The Vectara Zoo
               ○   Our on-site mini-zoo is home to these extraordinary animals.</b> It's a place of
                       inspiration, relaxation, and education. ○   Guided tours, animal interaction sessions, and wildlife workshops are regular
features. *score: 0.76677066, doc-id: vectara_employee_handbook.pdf*
 10. These shenanigans expose the
company to far too many potential legal issues. Furthermore, we've consulted with leading pet psychologists, animal behaviorists, and even a
few wizards to understand the complex dynamics between humans and their four-legged
companions. <b>The conclusion? Cats and dogs are simply too mainstream for Vectara's
avant-garde environment.</b> They're playing chess while we're orchestrating interdimensional
quantum maneuvers. We also considered the potential distractions. *score: 0.7343332, doc-id: vectara_employee_handbook.pdf*
